In [1]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup

## 1. Web scraping: Extraer lista: Los 100 mejores artistas de la Historia según la revista Rolling Stone

In [2]:
# Request a la web

url = 'https://es.wikipedia.org/wiki/Anexo:Los_100_mejores_artistas_de_la_Historia_seg%C3%BAn_la_revista_Rolling_Stone'
html = requests.get(url).content
soup = BeautifulSoup(html, 'html.parser')

In [3]:
# Extrar la tabla con los artistas y el pais de procedencia

table = soup.find_all('table',{'class':'wikitable'})[0]

rows = table.find_all('tr')
rows = [row.text.strip().split("\n") for row in rows]
rows.pop(0)
for row in rows:
    row.pop(0) and row.pop(0)

In [41]:
# Crear la tabla con pandas
colnames = ['artist','country_proc']
data = rows

music_artist = pd.DataFrame(data, columns=colnames)
country= list(music_artist['country_proc'])

country = [x.replace('Reino Unido\xa0Reino Unido','Reino Unido')
           .replace('Estados Unidos\xa0Estados Unidos','Estados Unidos')
           .replace('Jamaica\xa0Jamaica','Jamaica').replace('Irlanda\xa0Irlanda','Irlanda')
           .replace('Canadá\xa0Canadá','Canadá').replace('Australia\xa0Australia','Australia')
           .replace('México México','México')for x in country]


music_artist['country_proc']=country

del country[63]  #Elimino este artista ('Phil Spektor') porque la API no encuentra su ID

artist_listnames = list(music_artist['artist'])

del artist_listnames[63]     # Elimino este artista ('Phil Spektor') porque la API no encuentra su ID

music_artist = music_artist[music_artist.artist!='Phil Spector']

## 2. Hacer peticiones a la API (songkick) para cada uno de los artistas extraidos de los eventos pasados.


Songkick --> Solicito la KEY y me la mandan: 2KYDpKnYhVp98ZmM

https://api.songkick.com/api/3.0/search/artists.json?apikey={your_api_key}&query={artist_name}
https://api.songkick.com/api/3.0/artists/{artist_id}/gigography.json?apikey={your_api_key}

In [5]:
# 1. Conseguir el ID de cada artista. 

artist_listnames  #lista de los 99 mejores artistas según Rolling Stone

def lista_IDs(artist_list):
    
    def all_id(artistName): #Extrae un json de cada artista
        url = 'https://api.songkick.com/api/3.0/search/artists.json?apikey=2KYDpKnYhVp98ZmM&query={}'.format(artistName)
        get_ids = requests.get(url)
        artist_ids = pd.DataFrame(get_ids.json())
        artist_ids = artist_ids.transpose()

        def flatten(artist_ids, col_list): #Extrae del json exclusivamente el ID
            for column in col_list:
                flattened = pd.DataFrame(dict(artist_ids[column])).transpose()
                columns = [str(col) for col in flattened.columns]
                flattened.columns = [column + '_' + colname for colname in columns]
                artist_ids = pd.concat([artist_ids, flattened], axis=1)
                artist_ids = artist_ids.drop(column, axis=1)
            return artist_ids

        nested_columns = ['results']
        nested_columns1 = ['results_artist']
        nested_columns2 = ['results_artist_0']
        artist_ids1 = flatten(artist_ids, nested_columns)
        artist_ids2 = flatten(artist_ids1, nested_columns1)
        artist_ids3 = flatten(artist_ids2, nested_columns2)
        artistIDs = int(artist_ids3['results_artist_0_id'])

        return artistIDs

    list_IDs=[]
    for artistName in artist_list:   #Añade todos los ID a una lista
        list_IDs.append(all_id(artistName))
    return list_IDs

In [6]:
lista_99id = lista_IDs(artist_listnames)
print(lista_99id)

[417271, 408511, 561119, 379603, 537268, 263306, 336531, 29436, 174814, 379726, 137124, 268425, 421966, 18833, 525345, 400904, 136309, 22056, 87540, 6416, 471162, 313388, 131344, 528689, 96319, 99871, 509644, 174408, 69685, 250695, 483720, 187529, 231557, 1980138, 237682, 363507, 84028, 227030, 468870, 48820, 477246, 156780, 556617, 488285, 166932, 19445, 94437, 293701, 371163, 2477001, 469904, 216784, 171533, 424369, 387667, 357843, 528035, 4444353, 224919, 264775, 331163, 310304, 353621, 442154, 360549, 119202, 499703, 2318862, 17635, 191953, 276130, 253846, 216928, 556496, 313547, 448835, 218428, 171964, 201754, 410174, 87957, 182968, 431970, 260631, 25793, 8910724, 173376, 411643, 8511083, 253438, 234875, 72434, 241755, 330423, 350002, 220219, 404404, 13322, 12353]


In [26]:
# 2. Con el ID extraido en la lista (list_IDs), procedo a extraer los conciertos de cada artista

lista_99id

def artist_df (artistID): # crea un df con los conciertos de cada artista
    
    def events_page (artistID): #Conciertos por pagina y por artista
        url = 'https://api.songkick.com/api/3.0/artists/{}/gigography.json?apikey=2KYDpKnYhVp98ZmM'.format(artistID)
        get_events = requests.get(url)
        all_concerts= get_events.json()
        events = all_concerts['resultsPage']['results']['event']
        events_per_page = []
        for event in events:
            each_event = event['type'],event['displayName'],event['location']['city'],event['start']['date']
            events_per_page.append(each_event)

        return events_per_page
    
    conciertos_artista = pd.DataFrame(events_page (artistID), columns=['type_event','name_event','city_event','date_event'])
    conciertos_artista['artist_id'] = artistID
    
    return conciertos_artista

def lista_definitiva(lista_ids):
    
    lista_conciertos=pd.DataFrame()

    for artist in lista_ids:
        lista_conciertos = lista_conciertos.append(artist_df(artist))
    return lista_conciertos

data_conciertos_best = lista_definitiva(lista_99id)


4254

## 3. Crear el data set definitivo, uniendo el web scraping y los datos de conciertos de la api 

In [58]:


music_artist['artist_id']= lista_99id

merge_tablas = pd.merge(music_artist, data_conciertos_best, on=['artist_id'])

conciertos_best = merge_tablas.drop(columns=['id_artist'])
conciertos_best.head()

,artist,country_proc,artist_id,type_event,name_event,city_event,date_event
0,The Beatles,Reino Unido,417271,Concert,"The Beatles at St Peter’s Church (June 7, 1957)","Woolton, UK",1957-06-07
1,The Beatles,Reino Unido,417271,Concert,"The Beatles at Rosebery (June 22, 1957)","Liverpool, UK",1957-06-22
2,The Beatles,Reino Unido,417271,Concert,"The Beatles at Cavern Club Liverpool (July 8, ...","Liverpool, UK",1957-07-08
3,The Beatles,Reino Unido,417271,Concert,"The Beatles at Wilson Hall (July 11, 1957)","Liverpool, UK",1957-07-11
4,The Beatles,Reino Unido,417271,Concert,"The Beatles at Wilson Hall (July 12, 1957)","Liverpool, UK",1957-07-12


In [66]:
conciertos_best.shape

(4254, 7)

In [59]:
conciertos_best.to_csv('conciertos_best.csv')

In [64]:
conciertos_best['city_event'].value_counts().head(10)

London, UK                429
New York, NY, US          265
San Francisco, CA, US     152
Chicago, IL, US            91
Los Angeles, CA, US        82
Calgary, AB, Canada        77
Toronto, ON, Canada        65
West Hollywood, CA, US     61
Detroit, MI, US            60
Boston, MA, US             57
Name: city_event, dtype: int64